In [1]:
import tflite_runtime.interpreter as tflite
import cv2
import numpy as np
import os

protoPath = os.path.sep.join("/home/suraj/tensorflow/Notebooks/Face Mask/Face mask detector/face_detection_model", "deploy.prototxt"])
modelPath = os.path.sep.join("/home/suraj/tensorflow/Notebooks/Face Mask/Face mask detector/face_detection_model", "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [2]:
path = "/home/suraj/tensorflow/Notebooks/Face Mask/Face mask detector/face_detection_model/"
protoPath = path + "deploy.prototxt"
modelPath = path + "res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [3]:
import imutils

In [4]:
! pwd

/home/suraj/tensorflow/Notebooks/Face Mask/Face mask detector


In [5]:
interpreter = tflite.Interpreter(model_path='/home/suraj/tensorflow/Notebooks/Face Mask/Face mask detector/EnR_mask_detector_2.tflite')
interpreter.allocate_tensors()

In [6]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
print(input_details,'\n\n', output_details)

[{'name': 'conv2d_1_input', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}] 

 [{'name': 'Identity', 'index': 28, 'shape': array([1, 2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}]


In [35]:
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

In [36]:
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

In [37]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.9928185 0.0071815]]


In [38]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
label_dict = {0:'MASK', 1:"NO MASK"}
color_dict = {0:(0,255,0), 1:(0,0,255)}

In [39]:
label = np.argmax(output_data, axis=1)[0]

In [15]:
from time import sleep

source= cv2.VideoCapture(0)
sleep(2)

img_size=224
while True:
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)
    
    for x, y, w ,h in faces:
        face_img = img[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (img_size,img_size))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, input_shape)
        reshaped = np.float32(reshaped)
        interpreter.set_tensor(input_details[0]['index'], reshaped)
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]['index'])
        
        label = np.argmax(result, axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h), color_dict[label], 2)
#         cv2.rectangle(img,(x,y-4),(x+w,y), color_dict[label], -1)
        cv2.putText(img, label_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        cv2.putText(img, str(round(100*np.max(result),2))+ "%", (x+100,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        
    
    cv2.imshow("LIVE", img)
    key = cv2.waitKey(1)
    
    if (key==27):
        break
    
cv2.destroyAllWindows()
source.release()

In [43]:
from time import sleep

source= cv2.VideoCapture(0)
sleep(2)

img_size=200
while True:
    ret, img = source.read()
    if ret:
        frame = imutils.resize(img, width=600)
        h, w = frame.shape[:2]
        imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=False, crop=False)

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        detector.setInput(imageBlob)
        detections = detector.forward()

        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # extract the face ROI
                face = frame[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]

                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue

                # construct a blob for the face ROI, then pass the blob
                # through our face embedding model to obtain the 128-d
                # quantification of the face
                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
    
                resized = cv2.resize(frame, (img_size,img_size))
                normalized = resized/255.0
                reshaped = np.reshape(normalized, input_shape)
                reshaped = np.float32(reshaped)
                interpreter.set_tensor(input_details[0]['index'], reshaped)
                interpreter.invoke()
                result = interpreter.get_tensor(output_details[0]['index'])
        
                label = np.argmax(result, axis=1)[0]
        
                cv2.rectangle(img,(startX, startY),(endX, endY), color_dict[label], 2)
                cv2.putText(img, label_dict[label], (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
                cv2.putText(img, str(round(100*np.max(result),2))+ "%", (startX+100,startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
                
                cv2.imshow("LIVE", img)
                key = cv2.waitKey(1)
    
        if (key==27):
            break
    
cv2.destroyAllWindows()
source.release()

In [40]:
input_details

[{'name': 'conv2d_1_input',
  'index': 0,
  'shape': array([  1, 200, 200,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0}}]

In [13]:
input_shape

array([  1, 224, 224,   3], dtype=int32)

In [19]:
label

0

In [20]:
output_data

array([[0.9936727 , 0.00632726]], dtype=float32)

In [21]:
np.max(result)

0.54341894

In [22]:
str(np.max(result))

'0.54341894'